# Week 9: Sentence Level Classification with BERT

Your goal this week is to train a classifier that can predict the CEFR level of any given sentence. In this notebook we will guide you through the process of using 🤗[Hugging Face](https://huggingface.co/) and its transformers library as the training framework, with [Pytorch](https://pytorch.org/) as the deep learning backend, but feel free to use [TensorFlow](https://www.tensorflow.org) if that's what you are more familiar with.

For this assignment we will provide a dataset containing sentences with the corresponding CEFR level, and you have to use BERT and train a sentence classifier with this dataset.

## Prepare your environment

As always, we highly recommend that you install all packages with a virtual environment manager, like [venv](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/) or [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html), to prevent version conflicts of different packages.  

### Install CUDA
Deep learning is a computionally extensive process. It takes lots of time if relying only on the CPU, especially when it's trained on a large dataset. That's why using GPU instead is generally recommended.  
To use GPU for computation, you have to install [CUDA toolkit](https://developer.nvidia.com/cuda-toolkit) as well as the [cuDNN library](https://developer.nvidia.com/cudnn) provided by NVIDIA.  

If you already had CUDA installed on your machine, then great! You're done here.  
If you don't, you can refer to [Appendix](#Appendix-1-Install-CUDA) to see how to do so.


### Install python packages
The following python packages will be used in this tutorial:

1. `numpy`: for matrix operation
2. `scikit-learn`: for label encoding
3. `datasets`: for data preparation
4. `transformers`: for model loading and finetuing
5. `pytorch`: the backend DL framework
  - Note that the pt version must support the CUDA version you've installed if you want to use GPU.

### Select GPU(s) for your backend

Skip this section if you have no intension of using GPU with tensorflow/pytorch.

In [ ]:
import os

# select your GPU. Note that this should be set before you load tensorflow or pytorch.
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# To use multiple GPUs, combine all GPU ID with commas
# e.g. >>> os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,3'

In [ ]:
import torch
# Check if any GPU is used
torch.cuda.is_available()


/home/xu0494akk/miniconda3/envs/py36/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Prepare the dataset

Before starting the training, we need to load and process our dataset - but wait, let's decide which model we want to use first.  

In the highly unlikely chance you've never heard of it, [BERT](https://arxiv.org/abs/1810.04805) (**B**idirectional **E**ncoder **R**epresentations from **T**ransformers) is a language model proposed by Google AI in 2018, and it's currently one of the most popular models used in NLP.  
You can learn more about it here:
- [BERT Explained: A Complete Guide with Theory and Tutorial](https://towardsml.com/2019/09/17/bert-explained-a-complete-guide-with-theory-and-tutorial/) by Samia, 2019.


However, we will not directly use BERT in this tutorial, because it's large and takes too long to train. Instead, we'll be using [DistilBert](https://medium.com/huggingface/distilbert-8cf3380435b5), a version of BERT that while light-weight, reserves 95% of its original accuracy.




In [ ]:
# the model you want to use. Available models can be found here: https://huggingface.co/models
MODEL_NAME = 'bert-base-uncased'

### Load data

Similar to the `transformers` library, `datasets` is also a package by huggingface. It contains many public datasets online and can help us with the data processing.  
We can use `load_dataset` function to read the input `.csv` file provided for this assignment.

Reference:
 - [Official datasets document](https://huggingface.co/docs/datasets)
 - [datasets.load_dataset](https://huggingface.co/docs/datasets/loading.html)

In [ ]:
# [ TODO ] load the data using the load_dataset function

from datasets import load_dataset
dataset = load_dataset("/home/xu0494akk/nlplab/data")


Using custom data configuration data-9169fe6d1c846a71
Reusing dataset csv (/home/xu0494akk/.cache/huggingface/datasets/csv/data-9169fe6d1c846a71/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 2/2 [00:00<00:00, 43.23it/s]


In [ ]:
print(dataset)
print(dataset['train'])
print(dataset['train'][:5])


DatasetDict({
    train: Dataset({
        features: ['text', 'level'],
        num_rows: 20720
    })
    test: Dataset({
        features: ['text', 'level'],
        num_rows: 2300
    })
})
Dataset({
    features: ['text', 'level'],
    num_rows: 20720
})
{'text': ['My mother is having her car repaired.', 'You can contact me by e-mail.', 'He had a break for the weekend, and he called me: "I am in London, so, if you want to see me, it\'s the time!"', "Research shows that 40 percent of the program's viewers are aged over 55.", "I'd guess she's about my age."], 'level': ['B1', 'A1', 'B1', 'B2', 'A1']}


### Preprocessing

As always, texts should be tokenized, embedded, and padded before being put into the model.  
But not to worry, there are libraries from huggingface to help with this, too.

#### Sentence processing

Different pre-trained language models may have their own preprocessing models, and that's why we should use the tokenizers trained along with that model. In our case, we are using distilBERT, so we should use the distilBERT tokenizer.  

With huggingface, loading different tokenizers is extremely easy: just import the AutoTokenizer from `transformers` and tell it what model you plan to use, and it will handle everything for you.

Reference:
 - [transformers.AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer)

In [ ]:
# [ TODO ] load the distilBERT tokenizer using AutoTokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
example = "An explosion has been heard by the CNN team in Kyiv, about 45 minutes after an air raid siren was sounded."

example_embeddings = tokenizer(example)
example_embeddings

{'input_ids': [101, 2019, 7738, 2038, 2042, 2657, 2011, 1996, 13229, 2136, 1999, 25669, 1010, 2055, 3429, 2781, 2044, 2019, 2250, 8118, 19558, 2001, 5015, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### Label processing

Our labels also need to be processed, so let's do that next.

For this tutorial, we'll use the OneHotEncoder provided by scikit-learn.

For now, just declare a new encoder and use `fit` to learn the data. Hint: you should still end up with 6 labels.

Documents:
 - [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder)

In [ ]:
# [ TODO ] declare a new encoder and let it learn from the dataset
import numpy as np
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)
encoder = encoder.fit(np.reshape(dataset['train']['level'], (-1, 1)))


In [ ]:
# check if you still have 6 labels
LABEL_COUNT = len(encoder.categories_[0])
print(LABEL_COUNT)

6


In [ ]:
print(encoder.categories_)
print(encoder.transform([['A1'],['A2'],['B1'],['B2'],['C1'], ['C2']]))

print(encoder.inverse_transform([[0, 0, 1, 0, 0, 0]]))

[array(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], dtype='<U2')]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[['B1']]


#### Process the data

To make things easier, we can write a function to process our dataset in batches. 

In [ ]:
def preprocess(dataslice):    
    """ Input: a batch of your dataset
        Example: { 'text': [['sentence1'], ['setence2'], ...],
                   'label': ['label1', 'label2', ...] }
    """
    # [ TODO ] use your tokenizor and encoder to get sentence embeddings and encoded labels
    embeddings = tokenizer(dataslice['text'])
    dataslice.update(embeddings)
    encoder = OneHotEncoder(sparse = False)
    encoder = encoder.fit(np.reshape(dataslice['level'], (-1, 1)))
    dataslice['label'] = encoder.transform(np.reshape(dataslice['level'], (-1, 1)))
    return dataslice

    
    """ Output: a batch of processed dataset
        Example: { 'input_ids': ...,
                   'attention_masks': ...,
                   'label': ... }
    """
   
  

In [ ]:
# map the function to the whole dataset
processed_data = dataset.map(preprocess,    # your processing function
                             batched = True # Process in batches so it can be faster
                            )

Loading cached processed dataset at /home/xu0494akk/.cache/huggingface/datasets/csv/data-9169fe6d1c846a71/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c9d053e9f1ac937b.arrow
Loading cached processed dataset at /home/xu0494akk/.cache/huggingface/datasets/csv/data-9169fe6d1c846a71/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-e08a8b074cfc54cb.arrow


In [ ]:
print(processed_data)
processed_data['train'][0]

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 20720
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 2300
    })
})


{'text': 'My mother is having her car repaired.',
 'level': 'B1',
 'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'label': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]}

### DataCollator

You might have noticed that we skipped padding the sentences. That's because we are going to do it during training.  

To do training-time processing, we can use the DataCollator Class provided by `transformers`. And guess what - transformers has a class that will handle padding for us, too!

 - [transformers.DataCollatorWithPadding](https://huggingface.co/docs/transformers/master/en/main_classes/data_collator#transformers.DataCollatorWithPadding)

In [ ]:
# [ TODO ] declare a collator to do padding during traning
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Training

Finally, we can move on to training.

### Preparation

We can load the pretrained model from `transformers`.  
Generally, you need to build your own model on top of BERT if you want to use BERT for some downstream tasks, but again, sequence classification is a popular topic. With the support from `transformers` library, it can be done in two lines of codes: 

1. Load `AutoModelForSequenceClassification` Class.
2. Load the pretrained model.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels = LABEL_COUNT)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#### Split train/val data

The `Dataset` class we prepared before has a `train_test_split` method. You can use it to split your (processed) dataset.

Document:
 - [datasets.Dataset - Sort, shuffle, select, split, and shard](https://huggingface.co/docs/datasets/process.html#sort-shuffle-select-split-and-shard)

In [ ]:
# [ TODO ] choose a validation size and split your data
train_val_dataset = processed_data['train'].train_test_split(test_size = 0.2)

In [ ]:
print(train_val_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 16576
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 4144
    })
})


#### Setup training parameters

We are using the TrainerAPI to do the training. Trainer is yet another utility provided by huggingface, which helps you train the model with ease.  

Document:
- [transformers.TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments)
- [transformers.Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
# [ TODO ] set and tune your training properties
OUTPUT_DIR = 'model'
LEARNING_RATE = 5e-5
BATCH_SIZE = 32
EPOCH = 5
training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCH,
    # you can set more parameters here if you want
)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels = LABEL_COUNT)

# now give all the information to a trainer
trainer = Trainer(
    model_init  = model_init,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_val_dataset['train'],
    eval_dataset = train_val_dataset['test'],
    tokenizer = tokenizer,
    # set your parameters here
)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/xu0494akk/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_att

### Training

This is the easy part. Simply ask the trainer to train the model for you!

In [ ]:
trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/xu0494akk/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_att

Step,Training Loss
500,0.362700
1000,0.266600
1500,0.178200
2000,0.116800
2500,0.075300


Saving model checkpoint to model/checkpoint-500
Configuration saved in model/checkpoint-500/config.json
Model weights saved in model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in model/checkpoint-500/tokenizer_config.json
Special tokens file saved in model/checkpoint-500/special_tokens_map.json
Saving model checkpoint to model/checkpoint-1000
Configuration saved in model/checkpoint-1000/config.json
Model weights saved in model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in model/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to model/checkpoint-1500
Configuration saved in model/checkpoint-1500/config.json
Model weights saved in model/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in model/checkpoint-1500/tokenizer_config.json
Special tokens file saved in model/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to model/checkpoint-2000

TrainOutput(global_step=2590, training_loss=0.19522105128608616, metrics={'train_runtime': 373.9007, 'train_samples_per_second': 221.663, 'train_steps_per_second': 6.927, 'total_flos': 1960461546408576.0, 'train_loss': 0.19522105128608616, 'epoch': 5.0})

### Save for future use

Hint: try using `save_pretrained`

In [ ]:
trained_model_name = f"batch{str(BATCH_SIZE)}_epoch{str(EPOCH)}_lrate_{str(LEARNING_RATE)}"


In [ ]:
# [ TODO ] practice saving your model for future use
model.save_pretrained(os.path.join('model', trained_model_name))

Configuration saved in model/batch32_epoch5_lrate_5e-05/config.json
Model weights saved in model/batch32_epoch5_lrate_5e-05/pytorch_model.bin


In [ ]:
torch.cuda.empty_cache()

## Prediction

Now we know exactly how to train a model, but how do we use it for predicting results?

### Load finetuned model

In [ ]:
# [ TODO ] load the model that you saved

from transformers import AutoModelForSequenceClassification
mymodel = AutoModelForSequenceClassification.from_pretrained(os.path.join('model', trained_model_name))

loading configuration file model/batch32_epoch5_lrate_5e-05/config.json
Model config BertConfig {
  "_name_or_path": "model/batch32_epoch5_lrate_5e-05",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_voca

### Get the prediction

Here are a few example sentences:

In [ ]:
examples = [
    # A2
    "Remember to write me a letter.",
    # B2
    "Strawberries and cream - a perfect combination.",
    "This so-called \"Perfect Evening\" was so disappointing, as well as discouraging us from coming to your Circle Theatre again.",
    # C1
    "Some may altogether give up their studies, which I think is a disastrous move.",
]

All we need to do is to transform them to embeddings, and then we can get predictions by calling your finetuned model.  

Since we don't have a DataCollator to pad the sentence and do the matrix transformation this time, we have to pad and transform the matrice on our own.

In [ ]:
# Transform the sentences into embeddings
input = tokenizer(examples, truncation=True, padding=True, return_tensors="pt")
# Get the output
logits = mymodel(**input).logits
logits

tensor([[ 0.0390,  0.2067,  0.3228, -0.1616, -0.5962, -0.3523],
        [ 0.0053,  0.3278,  0.2742, -0.0489, -0.5476, -0.3183],
        [ 0.1105,  0.1611,  0.3583, -0.1852, -0.5540, -0.3359],
        [ 0.0168,  0.0991,  0.3156, -0.1858, -0.5132, -0.2980]],
       grad_fn=<AddmmBackward0>)

Logits aren't very readable for us. Let's use softmax 
activation to transform them into more probability-like numbers.

In [ ]:
from torch import nn

predicts = nn.functional.softmax(logits, dim = -1)
predicts

tensor([[0.1807, 0.2137, 0.2400, 0.1478, 0.0957, 0.1222],
        [0.1685, 0.2326, 0.2205, 0.1596, 0.0969, 0.1219],
        [0.1911, 0.2011, 0.2449, 0.1422, 0.0984, 0.1223],
        [0.1796, 0.1950, 0.2421, 0.1466, 0.1057, 0.1311]],
       grad_fn=<SoftmaxBackward0>)

#### Transform logits back to labels

Now you've got the output. Write a function to map it back into labels!

In [ ]:
# [ TODO ] try to process the result
predicts = torch.argmax(predicts, dim=1)
predicts

tensor([2, 1, 2, 2])

## Evaluation

Let's see how you did!  
Load the testing data and calculate your accuracy.

We want you to calculate the three kinds of accuracy mentioned in the lecture, which will also be explained in the following section.

In [ ]:
predictions, labels, metrics = trainer.predict(processed_data["test"])
predictions = torch.from_numpy(predictions)
predictions_softmax = nn.functional.softmax(predictions, dim = -1)

result = torch.argmax(predictions_softmax,dim=1).to("cuda:0")
pred_labels = torch.nn.functional.one_hot(result)
pred_level = encoder.inverse_transform(pred_labels.cpu()).flatten()


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: level, text. If level, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2300
  Batch size = 32


pred_level ['C2' 'B2' 'B2' 'B2' 'B2' 'A2' 'B1' 'B2' 'B1' 'C2']
label ['C2' 'B2' 'B2' 'C2' 'C1' 'A2' 'B1' 'B2' 'A2' 'C1']
level ['C2', 'B2', 'B2', 'C2', 'C1', 'A2', 'B1', 'B2', 'A2', 'C1']


In [ ]:
for idx, (sent, level) in enumerate(zip(processed_data["test"]['text'], pred_level)):
    if idx >= 10: break
    print(f'{level}: {sent}') 

C2: No longer a remote, backward, unimportant country, it became a force to be reckoned with in Europe.
B2: Unfortunately he was too fast and I couldn't keep up with him.
B2: Most mushrooms are totally harmless, but some are poisonous.
B2: This provided solid evidence that he committed the crime.
B2: You can't just accept everything you read in the newspapers at face value.
A2: Remember to write me a letter.
B1: She has long blond hair and blue eyes. She has a good figure.
B2: Nowadays the aim in clothing is not just for covering and protecting ourselves.
B1: Take two tablets, three times a day.
C2: Well, you will be if you saw our slide show and talk - members can hardly forget that relaxing afternoon when we unfolded the sails on the lake and enjoyed the tranquility of the area.


### Six Level Accuracy

Exact accuracy is probably what you're most familiar with:

$
accuracy = \frac{\#exactly\:the\:same\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
                    ^  ^     ^
```

The six level accuracy is $\frac{3}{6} = 0.5$

As the requirement, <u>your exact accuracy should be higher than $0.5$</u>.

In [ ]:
# [ TODO ] calculate accuracy

ground_truth = processed_data["test"]['level']


correct = 0
for index, level in enumerate(pred_level):
    if level == ground_truth[index]:
        correct += 1

print(correct/len(pred_level))


0.5660869565217391


### Three Level Accuracy

Three Level Accuracy is used when you only want a more general sense of right or wrong.

$
accuracy = \frac{\#the\:same\:ABC\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
              ^     ^  ^     ^
```

The three level accuracy is $\frac{4}{6} = 0.667$

As the requirement, <u>your exact accuracy should be higher than $0.6$</u>.

In [ ]:
# [ TODO ] calculate accuracy
ground_truth = processed_data["test"]['level']


correct = 0
for index, level in enumerate(pred_level):
    if level[0] == ground_truth[index][0]:
        correct += 1

print(correct/len(pred_level))


0.7508695652173913


### Fuzzy accuracy

However, the level of a sentence is relatively subjective. Generally speaking, $\pm1$ errors are allowed in the real evaluation in linguistic area.  

For example, if the actual label is 'B1', we'll also consider the prediction 'right' if the model predicts 'B2' or 'A2'.

Hence, the fuzzy accuracy is

$
accuracy = \frac{\#good\:enough\:answers}{\#total}
$

Example:
```
Prediction:   0 1 2 3 4 5
Ground truth: 0 1 1 3 3 3
              ^ ^ ^ ^ ^
```

The fuzzy accuracy is $\frac{5}{6} = 0.833$

As the requirement, <u>your accuracy should be higher than $0.8$</u>.

In [ ]:
# [ TODO ] calculate accuracy
ground_truth = processed_data["test"]['level']
categories = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

correct = 0
for index, level in enumerate(pred_level):
    if abs(categories.index(ground_truth[index]) - categories.index(level)) <= 1:
        correct += 1

print(correct/len(pred_level))

0.8843478260869565


## TA's Note

Congratulations, you made it to the end of the tutorial! Make sure you make an appointment to show your work and turn in your finished assignment before next week's lesson. We will ask you to run your code, so double check that everything is working and that your model is saved. Don't worry if you didn't pass the evaluation requirements, you'll still get partial points for trying.

## Appendix 


<a name="Appendix-1-Install-CUDA"></a>

### Appendix 1 - Install CUDA

1. Check your GPU vs. CUDA compatibility:
   - [NVIDIA -> Your GPU Compute Capability](https://developer.nvidia.com/cuda-gpus) -> GeForce and TITAN Products
2. Check library vs. CUDA compatibility: 
   - Pytorch: [Previous PyTorch Versions](https://pytorch.org/get-started/previous-versions/)
   - Tensorflow: [Linux/MacOX](https://www.tensorflow.org/install/source#tested_build_configurations) or [Windows](https://www.tensorflow.org/install/source_windows#tested_build_configurations)
3. Note the highest CUDA version that fits your system.

#### >> for conda/mamba users

You can directly install CUDA library with the selected CUDA version.
1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. `conda/mamba install -c conda-forge cudatoolkit=${VERSION}`

#### >> for non-conda users

1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. Download and install [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit-archive)
3. Download and install [cuDNN Library](https://developer.nvidia.com/rdp/cudnn-archive)

### Appendix 2 - Further Readings

1. [Huggingface Official Tutorials](https://github.com/huggingface/notebooks/tree/master/examples)
2. How to use Bert with other downstream tasks: [How to use BERT from the Hugging Face transformer library](https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209): 
3. Training with pytorch backend: [transformers-tutorials](https://github.com/abhimishra91/transformers-tutorials)
4. A more complicated example that include manual data/training processing with Pytorch: [Transformers for Multi-Label Classification made simple](https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1)
5. [Text Classification with tensorflow](https://github.com/huggingface/notebooks/blob/master/examples/text_classification-tf.ipynb): tensorflow example